# IMU Feature Extraction Playground

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import scipy.stats as stats

import biopsykit as bp
from biopsykit.signals.imu.feature_extraction.static_sequences import static_sequence_features
from biopsykit.utils.time import timedelta_to_time

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close('all')
sns.set_theme(style='ticks')

## Load Example Dataset

In [ ]:
data_path = Path("../../../../Data/Sleep_Data")
export_path = Path("../../exports")

subject_id = "AB31R"

In [ ]:
sleep_times = pd.read_csv(export_path.joinpath("chronotype_sleep_endpoints_merged.csv"), index_col=['subject', 'night'])
wake_onset = sleep_times['wake_onset_time']

In [ ]:
data, fs = bp.io.nilspod.load_dataset_nilspod(sorted(data_path.joinpath(subject_id).glob("*bin"))[0])

### Data from the last hour

In [ ]:
time_start = timedelta_to_time(wake_onset - pd.Timedelta("1h"))
time_end = timedelta_to_time(wake_onset)

In [ ]:
data.between_time(time_start.loc[("AB31R", 0)], time_end.loc[("AB31R", 0)])

## Sleep Endpoints

In [ ]:
sleep_results = bp.sleep.sleep_endpoints.predict_pipeline(data, sampling_rate=fs, sleep_wake_scale_factor=0.1)
sleep_endpoints = sleep_results['sleep_endpoints']

In [ ]:
df_endpoints = bp.sleep.sleep_endpoints.endpoints_as_df(sleep_endpoints, subject_id)
df_endpoints

In [ ]:
fig, ax = bp.sleep.plotting.sleep_imu_plot(data, sleep_endpoints=sleep_endpoints, downsample_factor=fs, figsize=(10, 5))
fig.tight_layout()

In [ ]:
data_sleep = data.loc[sleep_endpoints['sleep_onset']:sleep_endpoints['wake_onset']]
data_gyr = data_sleep.filter(like="gyr")
data_acc = data_sleep.filter(like="acc")

In [ ]:
window_size = 10*fs
overlap = int(0.9 * window_size)

In [ ]:
ss = bp.signals.imu.get_static_sequences(data_gyr, threshold=500, window_samples=window_size, overlap_samples=overlap)

In [ ]:
ss_features = static_sequence_features(data_acc, ss, start=sleep_endpoints['sleep_onset'], end=sleep_endpoints['wake_onset'])

In [ ]:
ss_features

In [ ]:
data_gyr.last("1h")

In [ ]:
ss_last = bp.signals.imu.get_static_sequences(data_gyr.last("1h"), threshold=500, window_samples=window_size, overlap_samples=overlap)
ss_features_last = static_sequence_features(data_acc.last("1h"), ss_last, start=sleep_endpoints['sleep_onset'], end=sleep_endpoints['wake_onset'])
ss_features_last.columns = [c + '_last_hour' for c in ss_features_last.columns]
ss_features.join(ss_features_last)